# TP5 : Auto Encodeurs

## Human In The Loop
 - On a besoin de beaucoup de Labels pour entrainer des modèles
 - Ca coute du temps humain, travail répétitif et peu gratifiant, ça coute cher et ça crée des dérives
 - [Your Roomba is watching you](https://www.technologyreview.com/2022/12/19/1065306/roomba-irobot-robot-vacuums-artificial-intelligence-training-data-privacy/)

## Auto Encodeurs
 - Une idée pour contrer le problème de labelisation des images : Les autoencodeurs 
 - Application : Implémenter un autoencoder pour encoder des courbes et étudier les atouts de ces architectures. 

# Autoencoders

 - Principe : C'est un réseau de neurones profond auquel on fournit $X$ en entrée et qui sort $X$ en sortie
 - ...
 - $X = Id X$ ?
 - L'intéret arrive quand on applique certaines contraintes à notre réseau.
 - Les applications sont alors multiples :
     - Compression
     - Extractions d'embeddings (comme en NLP)
     - Debruitage
     - Détection d'anomalie
     - Génération d'image   
 - L'architecture classique que l'on obtient est la suivante :
     - Un Encodeur
     - La représentation latente $Z$ (plus petite que l'entrée $X$)
     - Un décodeur

![](https://upload.wikimedia.org/wikipedia/commons/2/28/Autoencoder_structure.png)
      
 - C'est une approche non supervisée car on compare la sortie à l'entrée et on veut qu'elles soient identiques
    - Pas besoin de label
    - Scalable!
    - On peut utiliser des techniques de Data Augmentation si on a peu de données et/ou si on souhaite obtenir des propriétés spécifiques
 

### Encodage de courbes



On va créer un auto encodeur qui aurra comme code un vecteur de dimension 1

Créez 2 réseaux:
 - Encoder : 
    - Les données en entrée seront de dimension *vector_size*
    - La donnée en sortie sera de dimension 1 
 - Decoder
    - Les données en entrée seront de dimension 1
    - La donnée en sortie sera de dimension *vector_size* 
    
Prennez l'architecture la plus simple possible dans ce 1 er cas

Entrainement du réseau !

Tracez la courbe d'erreur au cours de l'entrainement

Affichez sur un graphe l'input X et sa reconstruction par l'autoencodeur

Obtient-on $z = a$ ? 
- Si oui : Pourquoi ?
- Si non : Comment inclure cette contrainte ?

On recommence en incluant la contrainte $z = a$

Maintenant que se passe-t-il quand on présente au réseau une discrétisation d'une fonction du type $$ g(x) = \sin(b \pi x) $$

Codez la fonction gen_g, qui prend en paramètre la taille de la discrétisation, sample un b random et retourne le vecteur G associé à la discrétisation de $g(x)$.

Affichez sur un graphe, G et sa reconstruction par l'autoencodeur

Calculez les distances entre l'entrée et la sortie

Vous venez de découvrir l'un des usages des autoencodeurs : La détection d'anomalies

Est-ce que la même architecture peut encoder des fonctions affine *et* des sinus?

- Que changer ?
- Implémentez !

Maintenant dans l'entrainement incluez des fonctions affines et des sinus. 

Changez $z$ vous n'avez plus la contrainte dimension = 1.

Entrainez!

Avec cette architecture je veux maintenant que vous puissiez me dire si c'est un sinus ou une fonction affine :
 - On fait de la classification avec ce réseau